In [2]:
import pandas as pd
import numpy as np
import glob
import sys
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
py_file_location = "/content/drive/MyDrive"
sys.path.append(os.path.abspath(py_file_location))

In [84]:
from function_tools import *

In [85]:
path_files = r"/content/drive/MyDrive/Data_M/1M_MinStp_clusters/"

In [86]:
# récupérer les clusters 
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_file= glob.glob(f"{path_files}*")

In [87]:
list_file

['/content/drive/MyDrive/Data_M/1M_MinStp_clusters/som_100',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/km_25',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/km_100',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/som_400',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/som_225',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/km_50']

In [88]:
liste_clean , liste_brut = make_liste(list_file)

In [81]:
list_file

['/content/drive/MyDrive/Data_M/1M_MinStp_clusters/km_25',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/km_50',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/km_100',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/som_100',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/som_225',
 '/content/drive/MyDrive/Data_M/1M_MinStp_clusters/som_400']

In [ ]:
for i,n in enumerate(liste_clean[5]):
      print(n[-17:-14],i)

In [90]:
%%time
clean_cluster= matrice(liste_clean)

CPU times: user 1.9 s, sys: 1.06 s, total: 2.96 s
Wall time: 4.36 s


In [94]:
len(clean_cluster[5])

400

In [95]:
%%time
brut_cluster= matrice(liste_brut)

CPU times: user 3.77 s, sys: 2.3 s, total: 6.07 s
Wall time: 12.3 s


In [98]:
data = make_df(brut_cluster, clean_cluster)

In [ ]:
data[2][0].head()

data_km : 
1.   index 0 : km25
2.   index 1: km50
3.   index 2: km100

data_SOM : 
1.   index 3 : som100
2.   index 4: som225
3.   index 5: som400

In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [100]:
from gensim.models import KeyedVectors

In [101]:
# récupérer les embeddings entrainés sur nos données
w2v_min = KeyedVectors.load('/content/drive/MyDrive/Data_M/w2v/w2vec_model_d300_1M_MinStp')

In [102]:
#encode un message
def encode(msg):
    return np.mean([w2v_min[word] for word in msg if word in w2v_min] or [np.zeros(300)], axis = 0)

In [103]:
# Encoder une liste de messages 
def encoded(liste):
  w2v=[]
  for i in range(len(liste)):
    w2v.append(np.array([encode(msg.split()) for msg in liste[i]]))
  return w2v

Encodage des différents set de clusters

In [104]:
len(clean_cluster)

6

In [105]:
def liste_encoded(liste_cluster):
  dataset_encoded=[]
  for i in range(len(liste_cluster)):
    dataset_encoded.append(encoded(liste_cluster[i]))
  return dataset_encoded

In [106]:
dataset_encoded= liste_encoded(clean_cluster)

In [107]:
len(dataset_encoded[0])

25

Entrainement des différents set de clusters

In [108]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
if_model=IsolationForest(n_jobs=-1, random_state=42)


Créer une matrice qui stock les différents models d'IF correspondant aux différents clusters

In [109]:
def train_if (dataset):
  models=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      if_model=IsolationForest(n_jobs=-1, random_state=42)
      models[i].append(if_model.fit(dataset[i][n]))
  return models

Appliquer IF sur les clusters de Km

In [110]:
%%time
models = train_if(dataset_encoded)

CPU times: user 20min 10s, sys: 33.7 s, total: 20min 44s
Wall time: 12min 34s


Appliquer IF sur les clusters de SOM

Prédictions pour les différents modèles

In [111]:
def pred(dataset,model):
  preds=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      preds[i].append(model[i][n].predict(dataset[i][n]))
  return preds

In [112]:
pred= pred(dataset_encoded, models)

In [113]:
def add_pred_to_df(dataset,pred):
  for i in range(len(pred)):
    for n in range(len(pred[i])):
      dataset[i][n]['pred']= pred[i][n]

In [114]:
add_pred_to_df(data, pred)

In [115]:
data[0][0]

,tweet,clean_twt,pred
0,L’ogre danois bien sûr\n,ogre danois bien sûr\n,1
1,J aimerais bien l avoir 😁\n,aimerais bien avoir\n,1
2,Parce que vous les valez bien ☺️\n,valez bien\n,1
3,Il est bien\n,bien\n,1
4,"Elle doit être bien fatiguée d'avoir, depuis s...","être bien fatiguée d'avoir,, tête.\n",1
...,...,...,...
6559,"J’aime bien French Montana, depuis Datpiff. Il...","aime bien french montana, datpiff. super bien ...",1
6560,Il peut bien aller se faire....\n,peut bien aller faire ....\n,1
6561,Bien sur...\n,bien ...\n,1
6562,"Lache rien , j'aime bien les gens cohérent com...","lache, aime bien cohérent.\n",1


In [116]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
def outliers(df):
  outliers=[]
  for i in range(len(df)):
    outliers.append(df[i]['tweet'].loc[df[i]['pred']==-1])
  return outliers

In [117]:
liste_cluster=["km25","km50","km100","som100","som225","som400"]

In [119]:
def save_outliers(outliers,string):
  for i in range(len(outliers)):
    with open(f"drive/MyDrive/minstp/"+string+f"/outliers_{i}", 'w') as f:
      for line in outliers[i]:
        f.write(line)

In [120]:
def loop_outliers(data, liste):
  for i in range(len(data)):
    save_outliers(outliers(data[i]),liste[i])

In [121]:
loop_outliers(data, liste_cluster)